In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import pickle
import json
import cv2
import re
import gc

In [ ]:
tf.test.is_gpu_available()

In [ ]:
# make image dataloader using flow_from_dataframe
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# load data to extract labels
data_dir = '../mmhs150k/'
model_dir = 'models/'
tweet_dict = json.load(open(data_dir + 'MMHS150K_GT.json', 'r'))

In [ ]:
# load pretrained LSTM and CNN for text and images
from tensorflow.keras.models import load_model

lstm = load_model(model_dir + 'lstm.h5')
cnn = load_model(model_dir + 'cnn_weighted.h5')

print(lstm.summary())
print(cnn.summary())

In [ ]:
# create new models that are all but the last layer
from tensorflow.keras.models import Sequential, Model

text_net = Sequential()
for layer in lstm.layers[:-1]: text_net.add(layer)
print(text_net.summary())

img_net = Sequential()
for layer in cnn.layers[:-3]: img_net.add(layer)
print(img_net.summary())

# free up memory from old models:
del lstm
del cnn
gc.collect()

In [ ]:
# build full FCM model using concatenation layer
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

text_input = Input((text_net.layers[0].input_shape[-1],)) # get rid of None's in front
img_text_input = Input((text_net.layers[0].input_shape[-1],))
img_input = Input((img_net.layers[0].input_shape[1:])) # get rid of None in front

text_embed = text_net(text_input)
img_text_embed = text_net(img_text_input)
img_embed = img_net(img_input)

x = concatenate([text_embed, img_text_embed, img_embed])
x = Dense(2048 + 150 + 150, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

fcm_model = Model(inputs=[text_input, img_text_input, img_input], outputs=prediction)
print(fcm_model.summary())

optimizer = Adam(lr = 1e-6)
fcm_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# method for cleaning text like in https://nlp.stanford.edu/projects/glove/preprocess-twitter.rb
def hashtag(text):
    hashtag_body = text.group()[1:]
    if hashtag_body.isupper(): return "<hashtag> {} ".format(hashtag_body.lower())
    else: return ' '.join(["<hashtag>"] + [re.sub(r"([A-Z])",r" \1", hashtag_body, flags=re.MULTILINE | re.DOTALL)])

def allcaps(text): return text.group().lower() + ' <allcaps> '    

def clean_tweet_text(t):
    eyes = r'[8:=;]'
    nose = r"['`\-]?"
    
    t = re.sub(r'https?:\/\/\S+\b|www\.(\w+\.)+\S*', '<url>', t)
    t = re.sub(r'@\w+', '<user>', t)
    t = re.sub(r'{}{}[)dD]+|[)dD]+{}{}'.format(eyes, nose, nose, eyes), '<smile>', t)
    t = re.sub(r'{}{}p+".format(eyes, nose)', '<lolface>', t)
    t = re.sub(r'{}{}\(+|\)+{}{}'.format(eyes, nose, nose, eyes), '<sadface>', t)
    t = re.sub(r'{}{}[\/|l*]'.format(eyes, nose), '<neutralface>', t)
    t = re.sub(r'/', ' / ', t)
    t = re.sub(r'<3','<heart>', t)
    t = re.sub(r'[-+]?[.\d]*[\d]+[:,.\d]*', '<number>', t)
    t = re.sub(r'#\S+', hashtag, t)
    t = re.sub(r'([!?.]){2,}', r'\1 <repeat>', t)
    t = re.sub(r'\b(\S*?)(.)\2{2,}\b', r'\1\2 <elong>', t)
    t = re.sub(r'([A-Z]){2,}', allcaps, t)
    t = re.sub(r'{}'.format(r'[\".,-;&:]'), ' ', t)
    return t.lower()

In [24]:
# custom data generator to handle multimodal data
from random import randint # for random cropping
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_len = text_net.layers[0].input_shape[-1]

class MMDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, splits_path, tweet_dict, tokenizer, pad_len, batch_size=32, dim=(299, 299), n_channels=3, 
                 shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.tweet_dict = tweet_dict
        self.tokenizer = tokenizer
        self.pad_len = pad_len
        
        # build labels list and id list
        self.id_list = open(splits_path, 'r').read().splitlines()
        self.labels = dict()
        for id in self.id_list:
            binary_labels = [1 if n > 0 else 0 for n in tweet_dict[id]['labels']]
            label = 1 if sum(binary_labels)/len(tweet_dict[id]['labels']) > 0.5 else 0
            self.labels[id] = label
        
        # create dictionary for embedded sequences (tuple of text and img_text)
        self.text_dict = self.process_text(self.id_list)
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.id_list) / self.batch_size)) + 1 # last batch is partial

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:index*self.batch_size + self.batch_size]
        
        
        # Find list of IDs
        id_list_temp = [self.id_list[k] for k in indexes]

        # Generate data
        X_txt, X_img_txt, X_img, y = self.__data_generation(id_list_temp)
        
        return [X_txt, X_img_txt, X_img], y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.id_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, id_list_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X_img = np.empty((len(id_list_temp), *self.dim, self.n_channels))
        X_txt = np.empty((len(id_list_temp), self.pad_len))
        X_img_txt = np.empty((len(id_list_temp), self.pad_len))
        y = np.empty(len(id_list_temp), dtype=int)

        # Generate data
        for i, ID in enumerate(id_list_temp):
            X_img[i,] = self.process_img(data_dir + 'img_resized/' + ID + '.jpg')
            X_txt[i,] = self.text_dict[ID][0]
            X_img_txt[i,] = self.text_dict[ID][1]

            # Store class
            y[i] = self.labels[ID]

        return X_txt, X_img_txt, X_img, y
    
    def process_img(self, path): # method for getting image
        img = Image.open(path)
        img.load()
        data = np.asarray(img, dtype='uint8')
        im = self.augment(data)
        
        if im.shape==(self.dim[0], self.dim[1]): im = np.stack((im,)*3, axis=-1) # handle grayscale
        
        return im
    
    def get_labels(self): # get list of labels for calculating AUROC
        return [self.labels[ID] for ID in self.id_list]
    
    def augment(self, im): # random crop and random mirror
        
        # random crop
        x_max, y_max = im.shape[0], im.shape[1]
        x_start, y_start = randint(0, x_max - self.dim[0]), randint(0, y_max - self.dim[1])
        im = im[x_start:x_start + self.dim[0], y_start:y_start + self.dim[1]]
        
        # random mirror
        if randint(0,1): im = np.flip(im, axis=1)
        
        return im
    
    def process_text(self, id_list):
        
        # matrix for texts
        texts = [clean_tweet_text(tweet_dict[ID]['tweet_text']) for ID in id_list]
        sequences = self.tokenizer.texts_to_sequences(texts)
        text_seqs = pad_sequences(sequences, maxlen=self.pad_len)
        
        # matrix for img_texts
        img_texts = []
        for ID in id_list:
            if os.path.exists(data_dir + 'img_txt/' + ID + '.json'):
                img_txt = json.load(open(data_dir + 'img_txt/' + ID + '.json', 'r'))['img_text']
                img_texts.append(img_txt)
            else: img_texts.append('')
        img_txt_sequences = self.tokenizer.texts_to_sequences(img_texts)
        img_text_seqs = pad_sequences(img_txt_sequences, maxlen=self.pad_len) 
        
        
        id_to_seq = dict() # map id to text sequence compatible with embedding layer
        for ID, txt, img_txt in zip(id_list, text_seqs, img_text_seqs):
            id_to_seq[ID] = (txt, img_txt)
        
        return id_to_seq

In [30]:
# create data generators
tokenizer = pickle.load(open(model_dir + 'tokenizer.pkl', 'rb'))
pad_len = text_net.layers[0].input_shape[-1]

train_gen = MMDataGenerator(splits_path=data_dir + 'splits/train_ids.txt',
                          tweet_dict=tweet_dict,
                          tokenizer=tokenizer,
                          pad_len=pad_len,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=True)

val_gen = MMDataGenerator(splits_path=data_dir + 'splits/val_ids.txt',
                          tweet_dict=tweet_dict,
                          tokenizer=tokenizer,
                          pad_len=pad_len,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=True)

test_gen = MMDataGenerator(splits_path=data_dir + 'splits/test_ids.txt',
                          tweet_dict=tweet_dict,
                          tokenizer=tokenizer,
                          pad_len=pad_len,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=False)

In [10]:
# train model
from tensorflow.keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint(model_dir + 'best_fcm.h5', save_best_only=True, monitor='val_accuracy', mode='max')

history = fcm_model.fit_generator(train_gen,
                    validation_data=val_gen,
                    shuffle=True,
                    epochs=8,
                    callbacks=[mcp_save])

Epoch 1/8
4214/4214 [==============================] - 7721s 2s/step - loss: 0.4840 - accuracy: 0.7849 - val_loss: 0.7733 - val_accuracy: 0.5454
Epoch 2/8
4214/4214 [==============================] - 7762s 2s/step - loss: 0.4558 - accuracy: 0.7948 - val_loss: 0.7617 - val_accuracy: 0.5782
Epoch 3/8
4214/4214 [==============================] - 7761s 2s/step - loss: 0.4498 - accuracy: 0.7998 - val_loss: 0.8254 - val_accuracy: 0.5554
Epoch 4/8
4214/4214 [==============================] - 7772s 2s/step - loss: 0.4479 - accuracy: 0.8006 - val_loss: 0.7508 - val_accuracy: 0.6038
Epoch 5/8
4214/4214 [==============================] - 8308s 2s/step - loss: 0.4454 - accuracy: 0.8016 - val_loss: 0.7600 - val_accuracy: 0.5966
Epoch 6/8
4214/4214 [==============================] - 7817s 2s/step - loss: 0.4444 - accuracy: 0.8019 - val_loss: 0.7659 - val_accuracy: 0.5946
Epoch 7/8
4214/4214 [==============================] - 7638s 2s/step - loss: 0.4432 - accuracy: 0.8026 - val_loss: 0.7548 - val_ac

In [33]:
# test
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import math

y_test = test_gen.get_labels()

# get AUROC
preds = fcm_model.predict_generator(test_gen)
print('Test AUROC:', roc_auc_score(y_test, preds))

# get loss and acc
preds_bin = np.array(preds)
preds_bin[preds>0.5] = 1
preds_bin[preds<=0.5] = 0
print('Test Accuracy:', accuracy_score(y_test, preds_bin))

# get F1
print('Test F1:', f1_score(y_test, preds_bin, zero_division=1))
print('Test Precision:', precision_score(y_test, preds_bin, zero_division=1))
print('Test Recall:', recall_score(y_test, preds_bin, zero_division=1))

Test AUROC: 0.7326197999999999
Test Accuracy: 0.6195
Test F1: 0.4586712192345996
Test Precision: 0.7944800394282898
Test Recall: 0.3224


In [38]:
# get loss and acc
preds_bin = np.array(preds)
preds_bin[preds>0.2] = 1
preds_bin[preds<=0.2] = 0
print('Test Accuracy:', accuracy_score(y_test, preds_bin))

# get F1
print('Test F1:', f1_score(y_test, preds_bin, zero_division=1))
print('Test Precision:', precision_score(y_test, preds_bin, zero_division=1))
print('Test Recall:', recall_score(y_test, preds_bin, zero_division=1))

Test Accuracy: 0.6752
Test F1: 0.6943346508563899
Test Precision: 0.6557056523284749
Test Recall: 0.7378


In [21]:
fcm_model.save(model_dir + 'fcm_with_weighted_cnn.h5')

In [ ]:
# test the data generator
X, ys = train_gen.__getitem__(0)
texts, img_texts, imgs = X
ids = train_gen.id_list[:32]

for ID, text, img_text, img, y in list(zip(ids, texts, img_texts, imgs, ys))[:3]:
    print(ID)
    print(text)
    print(img_text)
    img = Image.fromarray(np.uint8(img), 'RGB')
    display(img)
    print('label:', y, '\n\n\n')
